In [2]:
import warnings
warnings.filterwarnings('ignore', category=UserWarning)
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils import resample
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV, cross_val_predict
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import numpy as np
import pandas as pd
df1 = pd.read_csv(r"D:\desktop\B\数据\标准化后数据\df1.csv")
df2 = pd.read_excel(r"D:\desktop\B\数据\标准化后数据\df2.xlsx")
df1[['mort', 'CONS','MV','CRRT']] = df1[['mort',  'CONS','MV','CRRT']].astype('category')
df2[['mort',  'CONS','MV','CRRT']] = df2[['mort',  'CONS','MV','CRRT']].astype('category')
selected_features = ["CONS", "LDH", "MV", "AST", "CRRT", "U", "L", "HR", "D.Dimer", "CR"]
target = 'mort'
X = df1[selected_features]
y = df1[target]
X_train, X_internal_test, y_train, y_internal_test = train_test_split(X, y, test_size=0.3, random_state=42)
X_external_test = df2[selected_features]
y_external_test = df2[target]

In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_predict
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils import resample

RANDOM_SEED = 42

# 定义计算置信区间的函数
def bootstrap_metric_ci(y_true, y_pred, metric_func, n_bootstraps=10000, ci=95, random_state=42):
    rng = np.random.RandomState(random_state)
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    bootstrapped_scores = []
    
    for _ in range(n_bootstraps):
        indices = resample(np.arange(len(y_true)), replace=True, random_state=rng)
        score = metric_func(y_true[indices], y_pred[indices])
        bootstrapped_scores.append(score)
    
    lower_bound = np.percentile(bootstrapped_scores, (100-ci)/2)
    upper_bound = np.percentile(bootstrapped_scores, 100-(100-ci)/2)
    return lower_bound, upper_bound


params_rf = {
    'max_depth': [10, 13, 15, 20],
    'n_estimators': [100, 142, 200, 250],
    'min_samples_leaf': [1, 2, 4, 6]
}

best_models_rf = {}

# 逐步添加特征并训练模型
for i in range(1, len(selected_features) + 1):
    current_features = selected_features[:i]
    X_train_current = X_train[current_features]
    X_internal_test_current = X_internal_test[current_features]
    X_external_test_current = X_external_test[current_features]
    
    model_rf = RandomForestClassifier(random_state=RANDOM_SEED)

    # 使用 GridSearchCV 进行超参数调优, scoring 选择 'roc_auc'
    grid = GridSearchCV(model_rf, params_rf, cv=10, scoring='roc_auc', n_jobs=-1)
    grid.fit(X_train_current, y_train)
    
    best_model_rf = grid.best_estimator_
    
    # 使用 cross_val_predict 进行交叉验证预测
    y_train_pred_cv = cross_val_predict(best_model_rf, X_train_current, y_train, cv=10, method='predict')
    y_train_proba_cv = cross_val_predict(best_model_rf, X_train_current, y_train, cv=10, method='predict_proba')[:, 1]
    
    # 在内部验证集上使用交叉验证预测
    y_internal_pred_cv = cross_val_predict(best_model_rf, X_internal_test_current, y_internal_test, cv=10, method='predict')
    y_internal_proba_cv = cross_val_predict(best_model_rf, X_internal_test_current, y_internal_test, cv=10, method='predict_proba')[:, 1]
    
    # 在外部验证集上进行预测（外部验证集没有交叉验证）
    y_external_pred_rf = best_model_rf.predict(X_external_test_current)
    y_external_proba_rf = best_model_rf.predict_proba(X_external_test_current)[:, 1]
    
    # 计算训练集的各项指标及其置信区间
    train_accuracy = accuracy_score(y_train, y_train_pred_cv)
    train_precision = precision_score(y_train, y_train_pred_cv, average='weighted', zero_division=0)
    train_recall = recall_score(y_train, y_train_pred_cv, average='weighted')
    train_f1 = f1_score(y_train, y_train_pred_cv, average='weighted')
    train_auc = roc_auc_score(y_train, y_train_proba_cv)

    train_ci = {
        'accuracy_ci': bootstrap_metric_ci(y_train, y_train_pred_cv, accuracy_score),
        'precision_ci': bootstrap_metric_ci(y_train, y_train_pred_cv, lambda y_true, y_pred: precision_score(y_true, y_pred, average='weighted', zero_division=0)),
        'recall_ci': bootstrap_metric_ci(y_train, y_train_pred_cv, lambda y_true, y_pred: recall_score(y_true, y_pred, average='weighted')),
        'f1_score_ci': bootstrap_metric_ci(y_train, y_train_pred_cv, lambda y_true, y_pred: f1_score(y_true, y_pred, average='weighted')),
        'auc_ci': bootstrap_metric_ci(y_train, y_train_proba_cv, roc_auc_score)
    }

    # 计算内部验证集的各项指标及其置信区间
    internal_accuracy = accuracy_score(y_internal_test, y_internal_pred_cv)
    internal_precision = precision_score(y_internal_test, y_internal_pred_cv, average='weighted', zero_division=0)
    internal_recall = recall_score(y_internal_test, y_internal_pred_cv, average='weighted')
    internal_f1 = f1_score(y_internal_test, y_internal_pred_cv, average='weighted')
    internal_auc = roc_auc_score(y_internal_test, y_internal_proba_cv)

    internal_ci = {
        'accuracy_ci': bootstrap_metric_ci(y_internal_test, y_internal_pred_cv, accuracy_score),
        'precision_ci': bootstrap_metric_ci(y_internal_test, y_internal_pred_cv, lambda y_true, y_pred: precision_score(y_true, y_pred, average='weighted', zero_division=0)),
        'recall_ci': bootstrap_metric_ci(y_internal_test, y_internal_pred_cv, lambda y_true, y_pred: recall_score(y_true, y_pred, average='weighted')),
        'f1_score_ci': bootstrap_metric_ci(y_internal_test, y_internal_pred_cv, lambda y_true, y_pred: f1_score(y_true, y_pred, average='weighted')),
        'auc_ci': bootstrap_metric_ci(y_internal_test, y_internal_proba_cv, roc_auc_score)
    }

    # 计算外部验证集的各项指标及其置信区间
    external_accuracy = accuracy_score(y_external_test, y_external_pred_rf)
    external_precision = precision_score(y_external_test, y_external_pred_rf, average='weighted', zero_division=0)
    external_recall = recall_score(y_external_test, y_external_pred_rf, average='weighted')
    external_f1 = f1_score(y_external_test, y_external_pred_rf, average='weighted')
    external_auc = roc_auc_score(y_external_test, y_external_proba_rf)

    external_ci = {
        'accuracy_ci': bootstrap_metric_ci(y_external_test, y_external_pred_rf, accuracy_score),
        'precision_ci': bootstrap_metric_ci(y_external_test, y_external_pred_rf, lambda y_true, y_pred: precision_score(y_true, y_pred, average='weighted', zero_division=0)),
        'recall_ci': bootstrap_metric_ci(y_external_test, y_external_pred_rf, lambda y_true, y_pred: recall_score(y_true, y_pred, average='weighted')),
        'f1_score_ci': bootstrap_metric_ci(y_external_test, y_external_pred_rf, lambda y_true, y_pred: f1_score(y_true, y_pred, average='weighted')),
        'auc_ci': bootstrap_metric_ci(y_external_test, y_external_proba_rf, roc_auc_score)
    }

    best_models_rf[i] = {
        'features': current_features,
        'model': best_model_rf,  
        'train_metrics': {
            'accuracy': train_accuracy,
            'precision': train_precision,
            'recall': train_recall,
            'f1_score': train_f1,
            'auc': train_auc
        },
        'train_ci': train_ci,
        'internal_metrics': {
            'accuracy': internal_accuracy,
            'precision': internal_precision,
            'recall': internal_recall,
            'f1_score': internal_f1,
            'auc': internal_auc
        },
        'internal_ci': internal_ci,
        'external_metrics': {
            'accuracy': external_accuracy,
            'precision': external_precision,
            'recall': external_recall,
            'f1_score': external_f1,
            'auc': external_auc
        },
        'external_ci': external_ci,
        'best_params': grid.best_params_
    }

# 输出每个模型的结果
for i, results in best_models_rf.items():
    print(f"\nModel with top {i} features: {results['features']}")
    print("Best Parameters:", results['best_params'])

    print("Training Set Metrics:")
    for metric, value in results['train_metrics'].items():
        print(f"{metric.capitalize()}: {value:.4f} 置信区间: {results['train_ci'][metric+'_ci'][0]:.4f}-{results['train_ci'][metric+'_ci'][1]:.4f}")

    print("Internal Validation Set Metrics:")
    for metric, value in results['internal_metrics'].items():
        print(f"{metric.capitalize()}: {value:.4f} 置信区间: {results['internal_ci'][metric+'_ci'][0]:.4f}-{results['internal_ci'][metric+'_ci'][1]:.4f}")
    
    print("External Validation Set Metrics:")
    for metric, value in results['external_metrics'].items():
        print(f"{metric.capitalize()}: {value:.4f} 置信区间: {results['external_ci'][metric+'_ci'][0]:.4f}-{results['external_ci'][metric+'_ci'][1]:.4f}")



Model with top 1 features: ['CONS']
Best Parameters: {'max_depth': 10, 'min_samples_leaf': 1, 'n_estimators': 100}
Training Set Metrics:
Accuracy: 0.8676 置信区间: 0.8186-0.9118
Precision: 0.8760 置信区间: 0.8312-0.9185
Recall: 0.8676 置信区间: 0.8186-0.9118
F1_score: 0.8706 置信区间: 0.8235-0.9134
Auc: 0.7919 置信区间: 0.7022-0.8718
Internal Validation Set Metrics:
Accuracy: 0.7500 置信区间: 0.6591-0.8409
Precision: 0.8013 置信区间: 0.7206-0.8782
Recall: 0.7500 置信区间: 0.6591-0.8409
F1_score: 0.7621 置信区间: 0.6756-0.8452
Auc: 0.6758 置信区间: 0.5337-0.8096
External Validation Set Metrics:
Accuracy: 0.8558 置信区间: 0.7885-0.9231
Precision: 0.8503 置信区间: 0.7767-0.9215
Recall: 0.8558 置信区间: 0.7885-0.9231
F1_score: 0.8476 置信区间: 0.7696-0.9183
Auc: 0.7628 置信区间: 0.6643-0.8610

Model with top 2 features: ['CONS', 'LDH']
Best Parameters: {'max_depth': 10, 'min_samples_leaf': 6, 'n_estimators': 100}
Training Set Metrics:
Accuracy: 0.8431 置信区间: 0.7941-0.8922
Precision: 0.8431 置信区间: 0.7909-0.8938
Recall: 0.8431 置信区间: 0.7941-0.8922
F1_s

In [5]:
import pandas as pd
import os

# 设置输出文件夹路径
output_folder = r"D:\desktop\B\结果\表格\内部外部验证结果"
os.makedirs(output_folder, exist_ok=True)  # 确保输出文件夹存在

# 用于存储结果的列表
output_data = []

# 遍历模型结果并将其组织为字典
for i, results in best_models_rf.items():
    model_name = f"Model with top {i} features"
    for metric in results['train_metrics'].keys():
        output_data.append({
            'Model': model_name,
            'Dataset': 'Training Set',
            'Metric': metric.capitalize(),
            'Value': results['train_metrics'][metric],
            'Confidence Interval': f"{results['train_ci'][metric+'_ci'][0]:.4f}-{results['train_ci'][metric+'_ci'][1]:.4f}"
        })
        output_data.append({
            'Model': model_name,
            'Dataset': 'Internal Validation Set',
            'Metric': metric.capitalize(),
            'Value': results['internal_metrics'][metric],
            'Confidence Interval': f"{results['internal_ci'][metric+'_ci'][0]:.4f}-{results['internal_ci'][metric+'_ci'][1]:.4f}"
        })
        output_data.append({
            'Model': model_name,
            'Dataset': 'External Validation Set',
            'Metric': metric.capitalize(),
            'Value': results['external_metrics'][metric],
            'Confidence Interval': f"{results['external_ci'][metric+'_ci'][0]:.4f}-{results['external_ci'][metric+'_ci'][1]:.4f}"
        })

# 将列表转换为 DataFrame
output_df = pd.DataFrame(output_data)

# 设置文件名
output_file = os.path.join(output_folder, "RF_Results.xlsx")

# 将 DataFrame 保存为 Excel 文件
output_df.to_excel(output_file, index=False)

print(f"结果已保存至：{output_file}")


结果已保存至：D:\desktop\B\结果\表格\内部外部验证结果\RF_Results.xlsx


In [4]:
import pandas as pd
import os
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.model_selection import cross_val_predict

RANDOM_SEED = 42

# 使用所有选定的最佳特征
selected_features = ['CONS', 'LDH', 'MV', 'AST', 'CRRT', 'U', 'L', 'HR', 'D.Dimer', 'CR']

# 使用最佳参数
best_params = {'max_depth': 10, 'min_samples_leaf': 4, 'n_estimators': 100}

# 创建随机森林模型实例
model_rf = RandomForestClassifier(random_state=RANDOM_SEED, **best_params)

# 训练模型
model_rf.fit(X_train[selected_features], y_train)

# 进行交叉验证预测（训练集和内部验证集）
y_train_pred_cv = cross_val_predict(model_rf, X_train[selected_features], y_train, cv=10, method='predict')
y_train_proba_cv = cross_val_predict(model_rf, X_train[selected_features], y_train, cv=10, method='predict_proba')[:, 1]

y_internal_pred_cv = cross_val_predict(model_rf, X_internal_test[selected_features], y_internal_test, cv=10, method='predict')
y_internal_proba_cv = cross_val_predict(model_rf, X_internal_test[selected_features], y_internal_test, cv=10, method='predict_proba')[:, 1]

# 对外部验证集进行单次预测
y_external_pred_rf = model_rf.predict(X_external_test[selected_features])
y_external_proba_rf = model_rf.predict_proba(X_external_test[selected_features])[:, 1]

# 输出训练集、内部验证集和外部验证集的指标
train_accuracy = accuracy_score(y_train, y_train_pred_cv)
train_precision = precision_score(y_train, y_train_pred_cv, average='weighted', zero_division=0)
train_recall = recall_score(y_train, y_train_pred_cv, average='weighted')
train_f1 = f1_score(y_train, y_train_pred_cv, average='weighted')
train_auc = roc_auc_score(y_train, y_train_proba_cv)

internal_accuracy = accuracy_score(y_internal_test, y_internal_pred_cv)
internal_precision = precision_score(y_internal_test, y_internal_pred_cv, average='weighted', zero_division=0)
internal_recall = recall_score(y_internal_test, y_internal_pred_cv, average='weighted')
internal_f1 = f1_score(y_internal_test, y_internal_pred_cv, average='weighted')
internal_auc = roc_auc_score(y_internal_test, y_internal_proba_cv)

external_accuracy = accuracy_score(y_external_test, y_external_pred_rf)
external_precision = precision_score(y_external_test, y_external_pred_rf, average='weighted', zero_division=0)
external_recall = recall_score(y_external_test, y_external_pred_rf, average='weighted')
external_f1 = f1_score(y_external_test, y_external_pred_rf, average='weighted')
external_auc = roc_auc_score(y_external_test, y_external_proba_rf)

# 输出结果
print(f"Model with top {len(selected_features)} features: {selected_features}")
print("Best Parameters:", best_params)

print("Training Set Metrics:")
print(f"Accuracy: {train_accuracy:.4f}")
print(f"Precision: {train_precision:.4f}")
print(f"Recall: {train_recall:.4f}")
print(f"F1 Score: {train_f1:.4f}")
print(f"AUC: {train_auc:.4f}")

print("\nInternal Validation Set Metrics:")
print(f"Accuracy: {internal_accuracy:.4f}")
print(f"Precision: {internal_precision:.4f}")
print(f"Recall: {internal_recall:.4f}")
print(f"F1 Score: {internal_f1:.4f}")
print(f"AUC: {internal_auc:.4f}")

print("\nExternal Validation Set Metrics:")
print(f"Accuracy: {external_accuracy:.4f}")
print(f"Precision: {external_precision:.4f}")
print(f"Recall: {external_recall:.4f}")
print(f"F1 Score: {external_f1:.4f}")
print(f"AUC: {external_auc:.4f}")


Model with top 10 features: ['CONS', 'LDH', 'MV', 'AST', 'CRRT', 'U', 'L', 'HR', 'D.Dimer', 'CR']
Best Parameters: {'max_depth': 10, 'min_samples_leaf': 4, 'n_estimators': 100}
Training Set Metrics:
Accuracy: 0.8873
Precision: 0.8837
Recall: 0.8873
F1 Score: 0.8841
AUC: 0.9193

Internal Validation Set Metrics:
Accuracy: 0.8409
Precision: 0.8355
Recall: 0.8409
F1 Score: 0.8331
AUC: 0.9108

External Validation Set Metrics:
Accuracy: 0.8942
Precision: 0.9073
Recall: 0.8942
F1 Score: 0.8835
AUC: 0.8940


In [5]:
import os

# 指定保存路径
save_dir = r'D:\desktop\B\结果\表格\RF真实标签和预测概率'

# 检查文件夹是否存在，如果不存在就创建
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

# 获取训练集的真实标签和预测概率
y_train_true = y_train.values
train_data = pd.DataFrame({
    'True_Label': y_train_true,
    'Predicted_Probability': y_train_proba_cv
})
train_data.to_csv(os.path.join(save_dir, 'train_labels_and_probabilities.csv'), index=False)

# 获取内部验证集的真实标签和预测概率
y_internal_true = y_internal_test.values
internal_data = pd.DataFrame({
    'True_Label': y_internal_true,
    'Predicted_Probability': y_internal_proba_cv
})
internal_data.to_csv(os.path.join(save_dir, 'internal_validation_labels_and_probabilities.csv'), index=False)

# 获取外部验证集的真实标签和预测概率
y_external_true = y_external_test.values
external_data = pd.DataFrame({
    'True_Label': y_external_true,
    'Predicted_Probability': y_external_proba_rf
})
external_data.to_csv(os.path.join(save_dir, 'external_validation_labels_and_probabilities.csv'), index=False)

print("训练集、内部验证集和外部验证集的标签和预测概率已保存到指定文件夹。")


训练集、内部验证集和外部验证集的标签和预测概率已保存到指定文件夹。


In [4]:
import pandas as pd
import os
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.model_selection import cross_val_predict

RANDOM_SEED = 42

# 使用所有选定的最佳特征
selected_features = ['CONS', 'LDH', 'MV', 'AST', 'CRRT', 'U', 'L']

# 使用最佳参数
best_params = {'max_depth': 10, 'min_samples_leaf': 6, 'n_estimators': 142}

# 创建随机森林模型实例
model_rf = RandomForestClassifier(random_state=RANDOM_SEED, **best_params)

# 训练模型
model_rf.fit(X_train[selected_features], y_train)

# 进行交叉验证预测（训练集和内部验证集）
y_train_pred_cv = cross_val_predict(model_rf, X_train[selected_features], y_train, cv=10, method='predict')
y_train_proba_cv = cross_val_predict(model_rf, X_train[selected_features], y_train, cv=10, method='predict_proba')[:, 1]

y_internal_pred_cv = cross_val_predict(model_rf, X_internal_test[selected_features], y_internal_test, cv=10, method='predict')
y_internal_proba_cv = cross_val_predict(model_rf, X_internal_test[selected_features], y_internal_test, cv=10, method='predict_proba')[:, 1]

# 对外部验证集进行单次预测
y_external_pred_rf = model_rf.predict(X_external_test[selected_features])
y_external_proba_rf = model_rf.predict_proba(X_external_test[selected_features])[:, 1]

# 输出训练集、内部验证集和外部验证集的指标
train_accuracy = accuracy_score(y_train, y_train_pred_cv)
train_precision = precision_score(y_train, y_train_pred_cv, average='weighted', zero_division=0)
train_recall = recall_score(y_train, y_train_pred_cv, average='weighted')
train_f1 = f1_score(y_train, y_train_pred_cv, average='weighted')
train_auc = roc_auc_score(y_train, y_train_proba_cv)

internal_accuracy = accuracy_score(y_internal_test, y_internal_pred_cv)
internal_precision = precision_score(y_internal_test, y_internal_pred_cv, average='weighted', zero_division=0)
internal_recall = recall_score(y_internal_test, y_internal_pred_cv, average='weighted')
internal_f1 = f1_score(y_internal_test, y_internal_pred_cv, average='weighted')
internal_auc = roc_auc_score(y_internal_test, y_internal_proba_cv)

external_accuracy = accuracy_score(y_external_test, y_external_pred_rf)
external_precision = precision_score(y_external_test, y_external_pred_rf, average='weighted', zero_division=0)
external_recall = recall_score(y_external_test, y_external_pred_rf, average='weighted')
external_f1 = f1_score(y_external_test, y_external_pred_rf, average='weighted')
external_auc = roc_auc_score(y_external_test, y_external_proba_rf)

# 输出结果
print(f"Model with top {len(selected_features)} features: {selected_features}")
print("Best Parameters:", best_params)

print("Training Set Metrics:")
print(f"Accuracy: {train_accuracy:.4f}")
print(f"Precision: {train_precision:.4f}")
print(f"Recall: {train_recall:.4f}")
print(f"F1 Score: {train_f1:.4f}")
print(f"AUC: {train_auc:.4f}")

print("\nInternal Validation Set Metrics:")
print(f"Accuracy: {internal_accuracy:.4f}")
print(f"Precision: {internal_precision:.4f}")
print(f"Recall: {internal_recall:.4f}")
print(f"F1 Score: {internal_f1:.4f}")
print(f"AUC: {internal_auc:.4f}")

print("\nExternal Validation Set Metrics:")
print(f"Accuracy: {external_accuracy:.4f}")
print(f"Precision: {external_precision:.4f}")
print(f"Recall: {external_recall:.4f}")
print(f"F1 Score: {external_f1:.4f}")
print(f"AUC: {external_auc:.4f}")


Model with top 7 features: ['CONS', 'LDH', 'MV', 'AST', 'CRRT', 'U', 'L']
Best Parameters: {'max_depth': 10, 'min_samples_leaf': 6, 'n_estimators': 142}
Training Set Metrics:
Accuracy: 0.8824
Precision: 0.8788
Recall: 0.8824
F1 Score: 0.8795
AUC: 0.9196

Internal Validation Set Metrics:
Accuracy: 0.8523
Precision: 0.8494
Recall: 0.8523
F1 Score: 0.8435
AUC: 0.9043

External Validation Set Metrics:
Accuracy: 0.8846
Precision: 0.8906
Recall: 0.8846
F1 Score: 0.8744
AUC: 0.9031


In [5]:
import os

# 指定保存路径
save_dir = r'D:\desktop\B\结果\表格\RF真实标签和预测概率（7）'

# 检查文件夹是否存在，如果不存在就创建
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

# 获取训练集的真实标签和预测概率
y_train_true = y_train.values
train_data = pd.DataFrame({
    'True_Label': y_train_true,
    'Predicted_Probability': y_train_proba_cv
})
train_data.to_csv(os.path.join(save_dir, 'train_labels_and_probabilities.csv'), index=False)

# 获取内部验证集的真实标签和预测概率
y_internal_true = y_internal_test.values
internal_data = pd.DataFrame({
    'True_Label': y_internal_true,
    'Predicted_Probability': y_internal_proba_cv
})
internal_data.to_csv(os.path.join(save_dir, 'internal_validation_labels_and_probabilities.csv'), index=False)

# 获取外部验证集的真实标签和预测概率
y_external_true = y_external_test.values
external_data = pd.DataFrame({
    'True_Label': y_external_true,
    'Predicted_Probability': y_external_proba_rf
})
external_data.to_csv(os.path.join(save_dir, 'external_validation_labels_and_probabilities.csv'), index=False)

print("训练集、内部验证集和外部验证集的标签和预测概率已保存到指定文件夹。")


训练集、内部验证集和外部验证集的标签和预测概率已保存到指定文件夹。
